<a href="https://colab.research.google.com/github/tiffanytang34/Age_Detection_Using_Facial_Image_with_CNN/blob/main/3_deep_learning_CNN_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import cv2
import os
from zipfile import ZipFile
import time
from datetime import datetime
import itertools

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras import utils
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

# Setting random seeds to reduce the amount of randomness in the neural net weights and results.
# The results may still not be exactly reproducible.
np.random.seed(42)
tf.random.set_seed(42)

In [4]:
# Checking the installed version of TensorFlow.
# The code in this notebook was written using TensorFlow version 2.8.0.

tf.__version__

'2.8.0'

**Ensure that the Runtime Type for this notebook is set to GPU**.

Run the code below to check for GPU. If a GPU device is not found, change the runtime type under *Runtime* &#10141; *Change runtime type* &#10141; *Hardware accelerator* &#10141; *GPU* and run the notebook from the beginning again.

In [5]:
# Testing to ensure GPU is being utilized.

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


## Deep Learning: Data Importing

In [6]:
# Unzipping the dataset file combined_faces.zip

combined_faces_zip_path = "/content/drive/My Drive/Age_Detection/combined_faces.zip"

with ZipFile(combined_faces_zip_path, 'r') as myzip:
    myzip.extractall()
    print('Done unzipping combined_faces.zip')

Done unzipping combined_faces.zip


In [7]:
# Importing the table showing the breakdown of age-ranges into classes.

combined_classes = pd.read_csv("/content/drive/My Drive/Age_Detection/combined_faces_classes_summary.csv")
combined_classes

,Class label,Age-ranges (classes),No. of images,Class balance (%)
0,0,1 - 2,3192,9.53
1,1,3 - 9,2816,8.41
2,2,10 - 20,3136,9.37
3,3,21 - 25,3474,10.37
4,4,26 - 27,3217,9.61
5,5,28 - 31,3063,9.15
6,6,32 - 36,3086,9.22
7,7,37 - 45,3207,9.58
8,8,46 - 54,2802,8.37
9,9,55 - 65,2796,8.35


In [8]:
# Importing the training and testing datasets to create tensors of images using the filename paths.

train_df = pd.read_csv("/content/drive/My Drive/Age_Detection/images_filenames_labels_train.csv")
test_df = pd.read_csv("/content/drive/My Drive/Age_Detection/images_filenames_labels_test.csv")

In [9]:
train_df.head()

,filename,age,target
0,/content/content/combined_faces/17_21.jpg,17,2
1,/content/content/combined_faces/9_285.jpg,9,1
2,/content/content/combined_faces/3_306.jpg,3,1
3,/content/content/combined_faces/40_314.jpg,40,7
4,/content/content/combined_faces/21_178.jpg,21,3


In [10]:
test_df.head()

,filename,age,target
0,/content/content/combined_faces/3_564.jpg,3,1
1,/content/content/combined_faces/41_114.jpg,41,7
2,/content/content/combined_faces/39_272.jpg,39,7
3,/content/content/combined_faces/33_90.jpg,33,6
4,/content/content/combined_faces/4_269.jpg,4,1


In [11]:
train_df.shape

(23440, 3)

In [12]:
test_df.shape

(10046, 3)

### TensorFlow Dataset Pipeline

I prepared the images dataset for the nueral network with  creating **dataset pipelines using TensorFlow's built .Dataset API**. This appraoch would significantly reduced RAM consumption, as the images are now only being loaded into the memory in defined batch sizes as and when they are needed by the neural network. 

In [13]:
# Converting the filenames and target class labels into lists for both train and test datasets.

train_filenames_list = list(train_df['filename'])
train_labels_list = list(train_df['target'])

test_filenames_list = list(test_df['filename'])
test_labels_list = list(test_df['target'])

In [14]:
len(train_filenames_list)

23440

In [15]:
# Creating tensorflow constants of filenames and labels for train and test datasets from the lists defined above.

train_filenames_tensor = tf.constant(train_filenames_list)
train_labels_tensor = tf.constant(train_labels_list)

test_filenames_tensor = tf.constant(test_filenames_list)
test_labels_tensor = tf.constant(test_labels_list)

In [16]:
# Defining the number of classes and a function to read, decode the image from given tensor and one-hot encode the image label class.

num_classes = 11

def parse_function(filename, label):
    
    image_string = tf.io.read_file(filename)
    image_decoded = tf.io.decode_jpeg(image_string, channels=1)    # channels=1 to convert to grayscale, channels=3 to convert to RGB.
    # image_resized = tf.image.resize(image_decoded, [200, 200])
    label = tf.one_hot(label, num_classes)

    return image_decoded, label

In [17]:
# Getting the dataset ready for the neural network.
# Using the tensor vectors defined above, accessing the images in the dataset and passing them through the function defined above.

train_dataset = tf.data.Dataset.from_tensor_slices((train_filenames_tensor, train_labels_tensor))
train_dataset = train_dataset.map(parse_function)
train_dataset = train_dataset.repeat(3)
train_dataset = train_dataset.batch(256)    # Same as batch_size hyperparameter in model.fit() below.

test_dataset = tf.data.Dataset.from_tensor_slices((test_filenames_tensor, test_labels_tensor))
test_dataset = test_dataset.map(parse_function)
test_dataset = test_dataset.repeat(3)
test_dataset = test_dataset.batch(256)    # Same as batch_size hyperparameter in model.fit() below.

## Deep Learning: Classification Modeling

### Initial Modeling with CNN

I first built a basic **Convolutional neural Network(CNN) model** that perform with reasonable accuracy on the given data and with the given number of total parameters. All facial images wil converted to grayscale as input.

In [18]:
# Defining the architecture of the sequential neural network.

cnn1 = Sequential()

cnn1.add(Conv2D(filters=8, kernel_size=3, activation='relu', input_shape=(200, 200, 1)))    # 3rd dim = 1 for grayscale images.
cnn1.add(Conv2D(filters=16, kernel_size=3, activation='relu'))
cnn1.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn1.add(MaxPooling2D(pool_size=(2,2)))

cnn1.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn1.add(MaxPooling2D(pool_size=(2,2)))

cnn1.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn1.add(MaxPooling2D(pool_size=(2,2)))

cnn1.add(GlobalAveragePooling2D())    # GlobalAveragePooling2D(), compared to Flatten(), gave better accuracy values, and significantly reduced over-fitting and the no. of parameters.
cnn1.add(Dense(20, activation='relu'))
cnn1.add(Dense(11, activation='softmax'))

cnn1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 198, 198, 8)       80        
                                                                 
 conv2d_1 (Conv2D)           (None, 196, 196, 16)      1168      
                                                                 
 conv2d_2 (Conv2D)           (None, 194, 194, 32)      4640      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 97, 97, 32)       0         
 )                                                               
                                                                 
 conv2d_3 (Conv2D)           (None, 95, 95, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 47, 47, 32)       0         
 2D)                                                    

In [19]:
# Compiling the neural network defined above.

cnn1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

I will use ***EarlyStopping*** as a callback while training the CNN model to monitor the validation loss so as **to avoid over-fitting**. *EarlyStopping* will stop the model from training for further epochs if the validation loss starts to increase continuously.

In [20]:
# Defining the early stop to monitor the validation loss to avoid overfitting.

early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=1, mode='auto')

In [22]:
# Fitting the model.

cnn1_history = cnn1.fit(train_dataset,
                        batch_size=256,
                        validation_data=test_dataset,
                        epochs=30,
                        callbacks=[early_stop]
                       # shuffle=False    # shuffle=False to reduce randomness and increase reproducibility
                       )

Epoch 1/30
275/275 [==============================] - 320s 1s/step - loss: 2.0746 - accuracy: 0.2443 - val_loss: 2.0041 - val_accuracy: 0.2644
Epoch 2/30
275/275 [==============================] - 318s 1s/step - loss: 1.9030 - accuracy: 0.2955 - val_loss: 1.8876 - val_accuracy: 0.3000
Epoch 3/30
275/275 [==============================] - 318s 1s/step - loss: 1.8083 - accuracy: 0.3233 - val_loss: 1.8466 - val_accuracy: 0.3135
Epoch 4/30
275/275 [==============================] - 316s 1s/step - loss: 1.7519 - accuracy: 0.3395 - val_loss: 1.8138 - val_accuracy: 0.3210
Epoch 5/30
275/275 [==============================] - 316s 1s/step - loss: 1.7148 - accuracy: 0.3500 - val_loss: 1.8057 - val_accuracy: 0.3259
Epoch 6/30
275/275 [==============================] - 317s 1s/step - loss: 1.6882 - accuracy: 0.3619 - val_loss: 1.7248 - val_accuracy: 0.3504
Epoch 7/30
275/275 [==============================] - 316s 1s/step - loss: 1.6593 - accuracy: 0.3713 - val_loss: 1.7080 - val_accuracy: 0.3558